In [ ]:
import os
import sys
import shutil
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

import cv2

In [ ]:
# indices to columns of the ground-truth matrix
key = 'camera ID frame left top width height worldX worldY feetX feetyY'.split()
key = {k: v for v, k in enumerate(key)}

# cameras are synshcornised with respect to camera #5 and these are frame offsets
start_frame_nums = [5543, 3607, 27244, 31182, 1, 22402, 18968, 46766]

# num pixels in the longer side of the original videos
original_longer = 1920

# relative path to the folder with all frames
frames_folder = 'frames'
seqs_folder = 'fancy_seqs'

In [ ]:
# path to the ground-truth file
gt_path = '../trainvalRaw.mat'
# path = '../trainval.mat'

# path to the folder with extracted frames
data_path = '../processed/camera2_2'

camera_num = 2
max_n_objects = 3

# region of interest as y, x, h, w in the downscaled image
roi = 13, 48, 64, 64

# fraction of object that needs to be within the roi to be counted as present
intersection_threshold = .05
assert 0. <= intersection_threshold <= 1.

# num pixels in the longer side of downscaled videos
downscaled_longer = 160

# maximum allowed number of consecutive empty frames in a sequence
max_empty_frames = 2

# minimum sequence length
min_seq_len = 10

In [ ]:
ratio = float(original_longer) / downscaled_longer
original_roi = ratio * np.asarray(roi)
start_frame_num = start_frame_nums[camera_num - 1]

frames_folder = os.path.join(data_path, frames_folder)
seqs_folder = os.path.join(data_path, seqs_folder)

In [ ]:
print 'Loading "{}"'.format(os.path.basename(gt_path))

if 'Raw' in gt_path:
    import h5py
    f = h5py.File(gt_path, 'r')
    data = f['trainData'][()].T
else:
    import scipy.io
    f = scipy.io.loadmat(gt_path)
    data = f['trainData']

In [ ]:
# Select data for the specified camera and shift frame nums by the starting frame num
camera_idx = np.equal(data[:, 0], camera_num)
frames, left, top, width, height = np.split(data[camera_idx, 2:7], 5, -1)

boxes = np.concatenate([top, left, height, width], -1)
# frames -= start_frame_num

objects = np.concatenate([frames, boxes], -1) # frame, bbox

print 'Total number of objects:', objects.shape[0]
print 'Frames from {} to {}'.format(frames.min(), frames.max())

In [ ]:
# Select objects that are only withing the region of interest
def intersection(bbox, roi):
    """Computes area of intersection between boxes in `bbox` and boxes in `roi`.
    Dimensions in both should be given as (y, x, h, w).

    :param bbox: np.array of shape [N, 4]
    :param roi: np.array of shape [4] or [k, 4] where k \in {1, N}
    :return: np.array of shape [N]
    """

    bbox = np.asarray(bbox)
    roi = np.asarray(roi)

    while len(roi.shape) < len(bbox.shape):
        roi = roi[np.newaxis, ...]

    y_top = np.maximum(bbox[..., 0], roi[..., 0])
    x_left = np.maximum(bbox[..., 1], roi[..., 1])
    y_bottom = np.minimum(bbox[..., 0] + bbox[..., 2], roi[..., 0] + roi[..., 2])
    x_right = np.minimum(bbox[..., 1] + bbox[..., 3], roi[..., 1] + roi[..., 3])

    invalid_x = np.less(x_right, x_left)
    invalid_y = np.less(y_bottom, y_top)
    invalid = np.logical_or(invalid_x, invalid_y)

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
#     print 'invalid', invalid.sum()
    intersection_area[invalid] = 0.
    return intersection_area


def within_roi(bbox, roi, threshold):
    intersection_area = intersection(bbox, roi)
    object_area = np.prod(bbox[..., 2:], -1)
    fraction_of_object_area_in_roi = intersection_area / (object_area + 1e-8)
#     print roi
#     print bbox[0]
#     print bbox[..., 2:].shape
#     print 'object area', object_area.mean()
#     print 'int area', intersection_area.mean()
#     print 'fraction', fraction_of_object_area_in_roi.mean()
    objects_within_roi = np.greater(fraction_of_object_area_in_roi, threshold)
    return objects_within_roi

objects_within_roi = within_roi(objects[..., 1:], original_roi, intersection_threshold)
objects = objects[objects_within_roi]

print 'Total number of objects within ROI:', objects.shape[0]

In [ ]:
print objects[..., 1:].max(0)
print objects[..., 1:].min(0)
print original_roi

In [ ]:
# Count objects in all frames
unique_frames, object_counts = np.unique(objects[..., 0], return_counts=True)

max_frame = int(objects[:, 0].max())
frames_and_counts = np.zeros((max_frame, 2), dtype=np.int32)
frames_and_counts[:, 0] = np.arange(1, max_frame + 1)
frames_and_counts[unique_frames.astype(np.int32) - 1, 1] = object_counts.astype(np.int32)

In [ ]:
# Find frames with up to `max_n_objects`
up_to_n_objects = np.less_equal(frames_and_counts[:, 1], max_n_objects)
no_objects = np.equal(frames_and_counts[:, 1], 0)


allowed_frames = frames_and_counts[up_to_n_objects, 0]
print 'There are {} frames where the number of objects is between 1 and {}.'.format(up_to_n_objects.sum() - no_objects.sum(), max_n_objects)
print 'There are {} frames with zero objects.'.format(no_objects.sum())

In [ ]:
# Find all frames we have
def find_frames(path):
    frames_paths = [os.path.join(path, p) for p in os.listdir(path) if p.endswith('jpeg')]
    frames = {int(p.split('_')[-1].split('.')[0]): p for p in frames_paths}
    return frames

frames = find_frames(frames_folder)
frame_nums = sorted(frames.keys())
print frame_nums[:10]

In [ ]:
allowed_and_present_frames = set(frame_nums).intersection(allowed_frames)
allowed_and_present_frames = np.asarray(sorted(list(allowed_and_present_frames)))

print 'There are {} present frames that have less than {} objects.'.format(len(allowed_and_present_frames), max_n_objects)

In [ ]:
# determine number of objects in all frames that we can use

object_counts_in_present_frames = frames_and_counts[allowed_and_present_frames - 1, 1]
first_frame, last_frame = allowed_and_present_frames[[0, -1]]

# Some of the present frames are not allowed, hence the difference in numbers between consecutive
# frames might be bigger. Here we find the minimum interval, which is the interval that we have frames as.
# We use it later to create a mask that tells us exactly which of our frames we can use.
frame_interval = (allowed_and_present_frames[1:] - allowed_and_present_frames[:-1]).min()
regularly_spaced_frames = np.arange(first_frame, last_frame+1, frame_interval)
counts = np.zeros_like(regularly_spaced_frames)

i = 0
for frame_num, obj_count in zip(allowed_and_present_frames, object_counts_in_present_frames):
    while regularly_spaced_frames[i] < frame_num:
        i += 1
        counts[i] = obj_count
        
regularly_spaced_frames_and_counts = np.stack((regularly_spaced_frames, counts), -1)

In [ ]:
# creates sequences
seqs = [] # a list of list of consecutive frame numbers

last_frame_was_empty = None
n_empty_frames = 0
seq = []
for i, (frame_num, obj_count) in enumerate(regularly_spaced_frames_and_counts):
    
    frame_is_empty = (obj_count == 0)
    if frame_is_empty and len(seq) > 0:
        if n_empty_frames < max_empty_frames:
            n_empty_frames += 1
            seq.append(frame_num)
        else:
            n_empty_frames = 0
            seqs.append(seq)
            seq = []
    else:
        if len(seq) == 0:
            if last_frame_was_empty:
                seq.append(last_frame)
                
        seq.append(frame_num)
        
    last_frame = frame_num
    last_frame_was_empty = frame_is_empty

In [ ]:
# prune sequences
for i in xrange(len(seqs) - 1, -1, -1):
    if len(seqs[i]) <= min_seq_len:
        del seqs[i]

In [ ]:
seq_lens = [len(seq) for seq in seqs]
print 'Created {} sequences:'.format(len(seqs))
print '\tmin seq_len =', min(seq_lens)
print '\tmean seq_len =', np.mean(seq_lens)
print '\tmedian seq_len =', np.median(seq_lens)
print '\tmax seq_len =', max(seq_lens)
print '\ttotal number of frames =', sum(seq_lens)

In [ ]:
# # Create sequences
# def mkdir_p(path):
#     if not os.path.exists(path):
#         os.makedirs(path)
        
# mkdir_p(seqs_folder)
# for i, seq in enumerate(seqs):
#     seq_folder = os.path.join(seqs_folder, '{:04d}'.format(i))
#     mkdir_p(seq_folder)
#     for img_num in seq:
#         src_img_path = frames[img_num]
#         dst_img_path = os.path.join(seq_folder, os.path.basename(src_img_path))
#         shutil.copy(src_img_path, dst_img_path)

In [ ]:
def rect(bbox, c=None, facecolor='none', label=None, ax=None, line_width=1):
    r = Rectangle((bbox[1], bbox[0]), bbox[3], bbox[2], linewidth=line_width,
                  edgecolor=c, facecolor=facecolor, label=label)

    if ax is not None:
        ax.add_patch(r)
    return r

In [ ]:
from matplotlib.patches import Rectangle


def plot_random_frame(ax, only_roi=False):
    
    n_objects = 0
    while n_objects == 0:
        frame = np.random.choice(regularly_spaced_frames_and_counts[:, 0])
        img_path = frames[frame]
        bboxes = objects[np.equal(objects[:, 0], frame)][:, 1:]
        objects_within_roi = within_roi(bboxes, original_roi, intersection_threshold)
#         print bboxes.shape[0], objects_within_roi.sum(),
        bboxes = bboxes[objects_within_roi]

        n_objects = bboxes.shape[0]
#         print n_objects

    bboxes /= ratio
    img = cv2.imread(img_path)
    
    if only_roi:
        bboxes[:, :2] -= np.asarray(roi[:2]).reshape((1, 2))
        y, x, h, w = roi
        img = img[y:y+h, x:x+w]
    
    ax.imshow(img)
    ax.set_title('n_objects = {}'.format(n_objects))
    if not only_roi:
        rect(roi, c='g', ax=ax)

    for bbox in bboxes:
#         print bbox
        rect(bbox, c='r', ax=ax)
        
fig, axes = plt.subplots(4, 4, figsize=(16, 9), sharex=True, sharey=True)
for ax in axes.flatten():
    plot_random_frame(ax)